In [51]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random
import io

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from sklearn.metrics import classification_report
from keras_preprocessing.text import tokenizer_from_json
import gensim.downloader
from sklearn.ensemble import RandomForestClassifier


In [52]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
#data = pd.read_csv(os.path.join('..','data', 'ffmpeg_data_reduced.csv'))
data = pd.read_csv(os.path.join('..','data', 'qemu_data_reduced.csv'))

In [4]:
np.max(data["length"])

314

In [5]:
word_counts = data["func"].apply(lambda x: len(x.split()))
max_length = word_counts.max()
print("Maximum number of words:", max_length)


Maximum number of words: 314


In [6]:
print(data["target"].value_counts())
max_length = data['length'].max()
print(max_length)

0    9136
1    6727
Name: target, dtype: int64
314


In [7]:
print(data.head())

  project  target                                               func  length
0    qemu       1  static void filter_mirror_setup(netfilterstate...      31
1    qemu       1  static void test_init(testdata *d)\r\n{\r\n   ...      86
2    qemu       1  static inline int64_t sub64(const int64_t a, c...      15
3    qemu       1  static int xen_9pfs_connect(struct xendevice *...     260
4    qemu       1  static void nbd_refresh_filename(blockdriverst...     149


In [8]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt tokenizer models if not already downloaded
nltk.download('punkt')

tokenized_list = [word_tokenize(sentence) for sentence in data["func"].tolist()]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
w2v_vectors = gensim.downloader.load('word2vec-google-news-300')
dim = 300

In [10]:
# Evaluation functions
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f1 = 2*((prec*rec)/(prec+rec+K.epsilon()))
    return f1

def f2_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f2 = 5*((prec*rec)/(4*prec+rec+K.epsilon()))
    return f2

In [ ]:
def vectorize(words, w2v_vectors, dim):
    words_vecs = [w2v_vectors[word] for word in words if word in w2v_vectors]
    if len(words_vecs) == 0:
        return np.zeros(dim)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [25]:
class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        anchor, positive, negative = inputs
        p_dist = K.sum(K.square(anchor-positive), axis=-1)
        n_dist = K.sum(K.square(anchor-negative), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)

In [1]:
from tensorflow.keras.losses import TripletSemiHardLoss

def buildMLP(n_inputs):
    nIn = n_inputs
    model = Sequential()
    #model.add(Dense(256, input_dim=nIn, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))#hidden
    model.add(Dense(256, input_dim=nIn))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))#hidden
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1)) #binary classification
    model.add(Activation('sigmoid'))#Output # sigmoid # softmax
    optimizer = optimizers.Adam(learning_rate=0.001) #, epsilon=1e-8, decay=1e-6
    model.compile(loss=TripletLoss(), optimizer=optimizer, metrics=[f1_metric]) # 'binary_crossentropy' # "sparse_categorical_crossentropy"
    # triplet loss a=0.5 b=0.001 g=0.5
    return model

ImportError: cannot import name 'TripletSemiHardLoss' from 'tensorflow.keras.losses' (D:\anaconda3\envs\tfgpu\lib\site-packages\tensorflow\keras\losses\__init__.py)

In [66]:
X = np.array(tokenized_list)
y = data["target"].values

############## cross validation
scores=['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'f2']
values = [np.array([]) for i in range(0, len(scores))]
score_dict = OrderedDict(zip(scores, values))
k=10
f=0
kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)

nb_epoch = 100
BS = 32
print("Training...")
milli_sec1 = int(round(time.time() * 1000))

for train_index, test_index in kfold.split(X, y):
    f = f + 1
    print('fold number= ',f)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    X_train = np.array([vectorize(sentence, w2v_vectors, dim) for sentence in X_train])
    X_test = np.array([vectorize(sentence, w2v_vectors, dim) for sentence in X_test])

#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)

#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)

    myModel = buildMLP(X_train.shape[1])

    print("model summary\m", myModel.summary())
    csv_logger = CSVLogger('log.csv', append=True, separator=',')
    es = EarlyStopping(monitor='val_f1_metric', mode='max', verbose=1, patience=10)
    mc = ModelCheckpoint('best_model.h5', monitor='val_f1_metric', mode='max', verbose=1, save_best_only=True)
    print(type(X_train))
    history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])

    #load best model
    #model = load_model('best_model.h5')
    myModel.load_weights("best_model.h5")

    scores = myModel.evaluate(X_test, Y_test, verbose=0)
    #predictions = myModel.predict_classes(X_test, verbose=0)
    predScores = myModel.predict(X_test)
    predictions = (predScores > 0.5).astype("int32")
    #predictions = predScores.argmax(axis=1)
    accuracy=accuracy_score(Y_test, predictions)
    precision=precision_score(Y_test, predictions)
    recall=recall_score(Y_test, predictions)
    f1=f1_score(Y_test, predictions)
    roc_auc=roc_auc_score(Y_test, predictions)
    f2=5*precision*recall / (4*precision+recall)
    print(confusion_matrix(Y_test, predictions, labels=[0, 1]))
    tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
    acc = ((tp+tn)/(tp+tn+fp+fn))
    print("Accuracy:%.2f%%"%(acc*100))
    print("Precision:%.2f%%"%(precision*100))
    print("Recall:%.2f%%"%(recall*100))
    print("F1 score:%.2f%%"%(f1*100))
    print("Roc_Auc score:%.2f%%"%(roc_auc*100))
    print("F2 score:%.2f%%"%(f2*100))
    print(classification_report(Y_test, predictions))
    del myModel
    score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
    score_dict['precision'] = np.append(score_dict['precision'], precision)
    score_dict['recall'] = np.append(score_dict['recall'], recall)
    score_dict['f1'] = np.append(score_dict['f1'], f1)
    score_dict['roc_auc'] = np.append(score_dict['roc_auc'], roc_auc)
    score_dict['f2'] = np.append(score_dict['f2'], f2)

milli_sec2 = int(round(time.time() * 1000))
print("Cross Validation is completed after", milli_sec2-milli_sec1)

print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))
print("roc_auc: %.2f%% (%.2f%%)" % (score_dict['roc_auc'].mean()*100, score_dict['roc_auc'].std()*100))
print("f2: %.2f%% (%.2f%%)" % (score_dict['f2'].mean()*100, score_dict['f2'].std()*100))

D:\anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


Training...
fold number=  1


NameError: name 'TripletLoss' is not defined

In [ ]:
# X = np.array(tokenized_list)
# y = data["target"].values

# ############## cross validation
# scores=['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'f2']
# values = [np.array([]) for i in range(0, len(scores))]
# score_dict = OrderedDict(zip(scores, values))
# k=10
# f=0
# kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)

# nb_epoch = 100
# BS = 32
# print("Training...")
# milli_sec1 = int(round(time.time() * 1000))

# for train_index, test_index in kfold.split(X, y):
#     f = f + 1
#     print('fold number= ',f)
#     X_train, X_test = X[train_index], X[test_index]
#     Y_train, Y_test = y[train_index], y[test_index]

#     X_train = np.array([vectorize(sentence, w2v_vectors, dim) for sentence in X_train])
#     X_test = np.array([vectorize(sentence, w2v_vectors, dim) for sentence in X_test])

# #         #sampling
# #         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
# #         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)

# #         #shuffle dataset
# #         X_resampled=pd.DataFrame(X_res)
# #         Y_resampled=pd.DataFrame(Y_res)
# #         newTrain=X_resampled.assign(Label=Y_resampled.values)
# #         newTrain = shuffle(newTrain,random_state=seed)
# #         X_train=np.array(newTrain.iloc[:, 0:-1 ])
# #         X_train=pd.DataFrame(X_train)
# #         Y_train=np.array(newTrain.iloc[:, -1 ])
# #         Y_train=pd.DataFrame(Y_train)

#     myModel = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
#     myModel.fit(X_train, Y_train.ravel())
    
#     predictions = myModel.predict(X_test)
#     accuracy=accuracy_score(Y_test, predictions)
#     precision=precision_score(Y_test, predictions)
#     recall=recall_score(Y_test, predictions)
#     f1=f1_score(Y_test, predictions)
#     roc_auc=roc_auc_score(Y_test, predictions)
#     f2=5*precision*recall / (4*precision+recall)
#     print(confusion_matrix(Y_test, predictions, labels=[0, 1]))
#     tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
#     acc = ((tp+tn)/(tp+tn+fp+fn))
#     print("Accuracy:%.2f%%"%(acc*100))
#     print("Precision:%.2f%%"%(precision*100))
#     print("Recall:%.2f%%"%(recall*100))
#     print("F1 score:%.2f%%"%(f1*100))
#     print("Roc_Auc score:%.2f%%"%(roc_auc*100))
#     print("F2 score:%.2f%%"%(f2*100))
#     print(classification_report(Y_test, predictions))
#     del myModel
#     score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
#     score_dict['precision'] = np.append(score_dict['precision'], precision)
#     score_dict['recall'] = np.append(score_dict['recall'], recall)
#     score_dict['f1'] = np.append(score_dict['f1'], f1)
#     score_dict['roc_auc'] = np.append(score_dict['roc_auc'], roc_auc)
#     score_dict['f2'] = np.append(score_dict['f2'], f2)

# milli_sec2 = int(round(time.time() * 1000))
# print("Cross Validation is completed after", milli_sec2-milli_sec1)

# print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
# print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
# print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
# print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))
# print("roc_auc: %.2f%% (%.2f%%)" % (score_dict['roc_auc'].mean()*100, score_dict['roc_auc'].std()*100))
# print("f2: %.2f%% (%.2f%%)" % (score_dict['f2'].mean()*100, score_dict['f2'].std()*100))